In [1]:
#@title import module
!pip install nsepy 
from nsepy import get_history as gh
import datetime as dt
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [2]:
#@title add data
df=pd.read_csv("data.csv")
print("Number of rows and columns:", df.shape)

Number of rows and columns: (223, 16)


In [3]:
df.head()

,text,domain,label,count_paragraph,average_words_in_paragraph,count_positive,count_negative,count_image,count_date,num_num_in_df,percent_num_start_1,numberRecordDomain,percentFraudDomain,average_words,noun,num_per_cha(%)
0,Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...,binhluan.biz,1,11,44.727273,30,26,0,0,7,0.285714,1,1.0,38.909091,415,1.159
1,Thủ tướng Nhật cúi đầu xin lỗi vì tinh thần ph...,www.ipick.vn,1,4,38.400000,19,11,0,1,0,0.000000,2,1.0,31.000000,161,0.792
2,Choáng! Cơ trưởng đeo khăn quàng quẩy banh nóc...,tintucqpvn.net,1,9,24.900000,13,13,0,0,1,0.000000,5,1.0,21.600000,213,1.623
3,Chưa bao giờ nhạc Kpop lại dễ hát đến thế!!!\n...,tintucqpvn.net,1,17,38.222222,43,22,0,0,1,0.000000,5,1.0,33.000000,621,0.194
4,"Đại học Hutech sẽ áp dụng cải cách ""Tiếq Việt""...",www.gioitreviet.net,1,40,48.365854,91,57,2,0,6,0.333333,1,1.0,40.634146,1714,0.239


In [4]:
df["domain"].unique()

array(['binhluan.biz', 'www.ipick.vn', 'tintucqpvn.net',
       'www.gioitreviet.net', 'giadinhtiepthi.com',
       'suckhoe.vnexpress.net', 'phapluat.news', 'autoxe.net',
       'thegioitre.vn', 'baoangiang.com.vn', 'thoibao.today',
       'haiduong.tintuc.vn', 'laodong.vn', 'sorry.vn', 'thoibao.de',
       'baonuocmy.com', 'tinvn.info', 'www.vietgiaitri.com/',
       'https://news.zing.vn', 'news.zing.vn', 'thanhnien.vn',
       'vnexpress.net', 'giaitri.vnexpress.net', 'thethao.tuoitre.vn',
       'tuoitre.vn', 'kinhdoanh.vnexpress.net', 'dulich.vnexpress.net',
       'sohoa.vnexpress.net', 'doisong.vnexpress.net', 'dantri.com.vn'],
      dtype=object)

In [5]:
#bỏ text, domain
df = df.drop('text', 1)
df = df.drop('domain', 1)
df.head()

C:\Users\MyPC\AppData\Local\Temp/ipykernel_15004/4027926715.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('text', 1)
C:\Users\MyPC\AppData\Local\Temp/ipykernel_15004/4027926715.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('domain', 1)


,label,count_paragraph,average_words_in_paragraph,count_positive,count_negative,count_image,count_date,num_num_in_df,percent_num_start_1,numberRecordDomain,percentFraudDomain,average_words,noun,num_per_cha(%)
0,1,11,44.727273,30,26,0,0,7,0.285714,1,1.0,38.909091,415,1.159
1,1,4,38.400000,19,11,0,1,0,0.000000,2,1.0,31.000000,161,0.792
2,1,9,24.900000,13,13,0,0,1,0.000000,5,1.0,21.600000,213,1.623
3,1,17,38.222222,43,22,0,0,1,0.000000,5,1.0,33.000000,621,0.194
4,1,40,48.365854,91,57,2,0,6,0.333333,1,1.0,40.634146,1714,0.239


In [6]:
#@title tạo data training, bỏ 25% số dòng để train chống overfit
data = df

In [7]:
data.shape

(223, 14)

In [8]:
data_train,data_test=train_test_split(data, test_size=0.25)

In [9]:
data_train.shape

(167, 14)

In [10]:
data_test.shape

(56, 14)

In [11]:
data.head()

,label,count_paragraph,average_words_in_paragraph,count_positive,count_negative,count_image,count_date,num_num_in_df,percent_num_start_1,numberRecordDomain,percentFraudDomain,average_words,noun,num_per_cha(%)
0,1,11,44.727273,30,26,0,0,7,0.285714,1,1.0,38.909091,415,1.159
1,1,4,38.400000,19,11,0,1,0,0.000000,2,1.0,31.000000,161,0.792
2,1,9,24.900000,13,13,0,0,1,0.000000,5,1.0,21.600000,213,1.623
3,1,17,38.222222,43,22,0,0,1,0.000000,5,1.0,33.000000,621,0.194
4,1,40,48.365854,91,57,2,0,6,0.333333,1,1.0,40.634146,1714,0.239


In [12]:
#@title tách cột label ra vào biến y, còn lại là biến X
# split into input (X) and output (y) variables
X_train =  data_train.iloc[:,1:14].values
y_train = data_train.iloc[:,0:1].values

In [13]:
#Xem X
X_train

array([[7.00000000e+00, 3.20000000e+01, 1.90000000e+01, ...,
        2.86250000e+01, 1.99000000e+02, 2.12800000e+00],
       [2.30000000e+01, 5.78750000e+01, 7.80000000e+01, ...,
        4.85000000e+01, 1.15000000e+03, 8.82000000e-01],
       [8.00000000e+00, 3.44444444e+01, 1.30000000e+01, ...,
        3.12222222e+01, 2.58000000e+02, 1.10800000e+00],
       ...,
       [6.00000000e+00, 2.88571429e+01, 1.50000000e+01, ...,
        2.47142857e+01, 1.60000000e+02, 5.32000000e-01],
       [8.00000000e+00, 3.46666667e+01, 1.50000000e+01, ...,
        2.94444444e+01, 2.62000000e+02, 4.40000000e-01],
       [8.00000000e+00, 2.86666667e+01, 1.50000000e+01, ...,
        2.57777778e+01, 2.15000000e+02, 1.18800000e+00]])

In [14]:
#Xem y
y_train

array([[0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
    

In [15]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=13, activation='relu'))
model.add(Dense(9, activation='relu'))
model.add(Dense(9, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [16]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=500, batch_size=10)

Epoch 1/500
17/17 [==============================] - 1s 1ms/step - loss: 31.9143 - accuracy: 0.4371
Epoch 2/500
17/17 [==============================] - 0s 1ms/step - loss: 19.4351 - accuracy: 0.4371
Epoch 3/500
17/17 [==============================] - 0s 1ms/step - loss: 8.6475 - accuracy: 0.4251
Epoch 4/500
17/17 [==============================] - 0s 1ms/step - loss: 2.3535 - accuracy: 0.4431
Epoch 5/500
17/17 [==============================] - 0s 2ms/step - loss: 1.5695 - accuracy: 0.4431
Epoch 6/500
17/17 [==============================] - 0s 2ms/step - loss: 1.3121 - accuracy: 0.4551
Epoch 7/500
17/17 [==============================] - 0s 1ms/step - loss: 1.1398 - accuracy: 0.4671
Epoch 8/500
17/17 [==============================] - 0s 1ms/step - loss: 1.0548 - accuracy: 0.4611
Epoch 9/500
17/17 [==============================] - 0s 1ms/step - loss: 0.9568 - accuracy: 0.4850
Epoch 10/500
17/17 [==============================] - 0s 1ms/step - loss: 0.8975 - accuracy: 0.4970
Epoch 1

17/17 [==============================] - 0s 1ms/step - loss: 0.4183 - accuracy: 0.8144
Epoch 83/500
17/17 [==============================] - 0s 1ms/step - loss: 0.4084 - accuracy: 0.8563
Epoch 84/500
17/17 [==============================] - 0s 1ms/step - loss: 0.4730 - accuracy: 0.8204
Epoch 85/500
17/17 [==============================] - 0s 1ms/step - loss: 0.3865 - accuracy: 0.8503
Epoch 86/500
17/17 [==============================] - 0s 1ms/step - loss: 0.4759 - accuracy: 0.7964
Epoch 87/500
17/17 [==============================] - 0s 1ms/step - loss: 0.4491 - accuracy: 0.8563
Epoch 88/500
17/17 [==============================] - 0s 873us/step - loss: 0.4323 - accuracy: 0.8024
Epoch 89/500
17/17 [==============================] - 0s 1ms/step - loss: 0.4049 - accuracy: 0.8503
Epoch 90/500
17/17 [==============================] - 0s 997us/step - loss: 0.5522 - accuracy: 0.7725
Epoch 91/500
17/17 [==============================] - 0s 997us/step - loss: 0.5237 - accuracy: 0.8024
Epoch 9

17/17 [==============================] - 0s 997us/step - loss: 0.2332 - accuracy: 0.9042
Epoch 163/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2427 - accuracy: 0.8982
Epoch 164/500
17/17 [==============================] - 0s 997us/step - loss: 0.2732 - accuracy: 0.8862
Epoch 165/500
17/17 [==============================] - 0s 935us/step - loss: 0.2510 - accuracy: 0.9162
Epoch 166/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2732 - accuracy: 0.8982
Epoch 167/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2411 - accuracy: 0.9162
Epoch 168/500
17/17 [==============================] - 0s 997us/step - loss: 0.2575 - accuracy: 0.8922
Epoch 169/500
17/17 [==============================] - 0s 997us/step - loss: 0.2179 - accuracy: 0.9102
Epoch 170/500
17/17 [==============================] - 0s 997us/step - loss: 0.2029 - accuracy: 0.9341
Epoch 171/500
17/17 [==============================] - 0s 1ms/step - loss: 0.2467 - accuracy:

17/17 [==============================] - 0s 1ms/step - loss: 0.1097 - accuracy: 0.9760
Epoch 243/500
17/17 [==============================] - 0s 1ms/step - loss: 0.1273 - accuracy: 0.9581
Epoch 244/500
17/17 [==============================] - 0s 1ms/step - loss: 0.1380 - accuracy: 0.9521
Epoch 245/500
17/17 [==============================] - 0s 997us/step - loss: 0.1819 - accuracy: 0.9222
Epoch 246/500
17/17 [==============================] - 0s 997us/step - loss: 0.1236 - accuracy: 0.9641
Epoch 247/500
17/17 [==============================] - 0s 997us/step - loss: 0.0964 - accuracy: 0.9760
Epoch 248/500
17/17 [==============================] - 0s 1ms/step - loss: 0.1342 - accuracy: 0.9521
Epoch 249/500
17/17 [==============================] - 0s 1ms/step - loss: 0.1673 - accuracy: 0.9461
Epoch 250/500
17/17 [==============================] - 0s 997us/step - loss: 0.1020 - accuracy: 0.9581
Epoch 251/500
17/17 [==============================] - 0s 997us/step - loss: 0.1265 - accuracy: 0

Epoch 323/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 0.9940
Epoch 324/500
17/17 [==============================] - 0s 997us/step - loss: 0.0335 - accuracy: 0.9940
Epoch 325/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0302 - accuracy: 1.0000
Epoch 326/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0283 - accuracy: 1.0000
Epoch 327/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0363 - accuracy: 0.9940
Epoch 328/500
17/17 [==============================] - 0s 1ms/step - loss: 0.1078 - accuracy: 0.9641
Epoch 329/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0315 - accuracy: 1.0000
Epoch 330/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0448 - accuracy: 0.9880
Epoch 331/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0317 - accuracy: 0.9940
Epoch 332/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0354 - accur

17/17 [==============================] - 0s 1ms/step - loss: 0.0152 - accuracy: 0.9940
Epoch 404/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 405/500
17/17 [==============================] - 0s 997us/step - loss: 0.0143 - accuracy: 1.0000
Epoch 406/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 407/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0100 - accuracy: 1.0000
Epoch 408/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 409/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 410/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 411/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0180 - accuracy: 0.9940
Epoch 412/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0676 - accuracy: 0.9820
Ep

17/17 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 485/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 486/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 487/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 488/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 489/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 490/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 491/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 492/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 493/500
17/17 [==============================] - 0s 1ms/step - loss: 0.0041 - accuracy: 1.0000
Epoc

In [18]:
#test 
X_test =  data_test.iloc[:,1:14].values
y_test = data_test.iloc[:,0:1].values

In [19]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

2/2 [==============================] - 0s 3ms/step - loss: 0.0406 - accuracy: 0.9821
Accuracy: 98.21


In [20]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [21]:
model.predict(np.array([[11.,24.33333333,13.,17.,0.,1.,13.,0.30769231,14.,0.,20.75,235.,1.972],]))

array([[0.00253308]], dtype=float32)

In [22]:
from tensorflow import keras
model2 = keras.models.load_model('model')
model2